In [2]:
from graphdatascience import GraphDataScience
from getpass import getpass

# Set up connections

In [3]:
vertex_ai_token = getpass()

 ········


In [4]:
neo4j_password = getpass()

 ········


In [5]:
neo4j_uri = "neo4j+s://bc9f751a.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Embed summaries using same model as documents

In [8]:
gds.run_cypher("""
CREATE VECTOR INDEX topic_group_summary_embeddings IF NOT EXISTS 
                  FOR (g:TopicGroup)
                  ON (g.summaryEmbedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 768,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""


In [11]:
gds.run_cypher("""
MATCH (g:TopicGroup)
WITH collect(g) AS groupNodes, collect(g.summary) AS summaries
CALL genai.vector.encodeBatch(summaries, "VertexAI", { token: $token, projectId: $project }) 
YIELD index, vector
WITH groupNodes[index] AS g, vector
CALL db.create.setNodeVectorProperty(g, 'summaryEmbedding', vector)""",
              {"token": vertex_ai_token, "project": "neo4j-ps-202001"})


""
